# Data Generation

## Dataset Description

Since we don't have real data, we've made a **synthetic dataset** that simulates user behavior based on realistic assumptions and probability distributions.


### Key Columns
- **user_id**: A unique identifier for each user.
- **group**: Either 'control' or 'experiment', indicating whether the user belongs to the control group or the experiment group.
- **session_date**: The date and time of the user's session.
- **product_views**: The number of products viewed by the user during the session.
- **cart_adds**: The number of items added to the cart.
- **purchase_amount**: The total amount spent by the user in the session (if any purchase was made).
- **session_duration**: The duration of the session in minutes.
- **device_type**: The type of device used by the user (mobile, desktop, or tablet).
- **traffic_source**: The source of traffic that brought the user to the site (organic, paid ad, or direct).
- **region**: The region where the user is located (Estonia, Latvia, Lithuania).
- **visitor_type**: Whether the user is a "new" or "old" visitor (new or returning customer).

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

# Function to generate data with an exponential distribution for purchase_amount
def generate_ab_test_data(n_control, n_experiment, start_date, end_date, control_conversion_rate, experiment_conversion_rate, control_arpu_mean, experiment_arpu_mean):
    # List of experiment days
    date_range = pd.date_range(start=start_date, end=end_date, freq='h')
    
    # Function to generate session dates
    def generate_session_dates(n_sessions):
        return np.random.choice(date_range, size=n_sessions, replace=True)

    # Probability distributions
    product_views_dist = np.random.poisson(5, size=n_control + n_experiment)
    cart_adds_dist = np.random.poisson(2, size=n_control + n_experiment)
    session_duration_dist = np.random.exponential(scale=10, size=n_control + n_experiment)

    # Distributions for device_type, traffic_source, region, visitor_type
    device_type_dist = np.random.choice(['mobile', 'desktop', 'tablet'], p=[0.7, 0.25, 0.05], size=n_control + n_experiment)
    traffic_source_dist = np.random.choice(['organic', 'paid_ad', 'direct'], p=[0.5, 0.3, 0.2], size=n_control + n_experiment)
    region_dist = np.random.choice(['Estonia', 'Latvia', 'Lithuania'], p=[0.3, 0.4, 0.3], size=n_control + n_experiment)
    
    # Adding distribution for visitor_type (e.g., 60% returning, 40% new)
    visitor_type_dist = np.random.choice(['new', 'old'], p=[0.4, 0.6], size=n_control + n_experiment)

    # Data generation for the control group
    control_data = {
        'user_id': np.arange(1, n_control + 1),
        'group': ['control'] * n_control,
        'session_date': generate_session_dates(n_control),
        'product_views': product_views_dist[:n_control],
        'cart_adds': cart_adds_dist[:n_control],
        'purchase_amount': np.where(np.random.rand(n_control) < control_conversion_rate,  # Conversion probability for the control group
                                    np.random.exponential(scale=control_arpu_mean / 2, size=n_control), 0),  # Exponential distribution for ARPU
        'session_duration': session_duration_dist[:n_control],
        'device_type': device_type_dist[:n_control],
        'traffic_source': traffic_source_dist[:n_control],
        'region': region_dist[:n_control],
        'visitor_type': visitor_type_dist[:n_control]
    }

    # Data generation for the experiment group
    experiment_data = {
        'user_id': np.arange(n_control + 1, n_control + n_experiment + 1),
        'group': ['experiment'] * n_experiment,
        'session_date': generate_session_dates(n_experiment),
        'product_views': product_views_dist[n_control:],
        'cart_adds': cart_adds_dist[n_control:],
        'purchase_amount': np.where(np.random.rand(n_experiment) < experiment_conversion_rate,  # Conversion probability for the experiment group
                                    np.random.exponential(scale=experiment_arpu_mean / 2, size=n_experiment), 0),  # Exponential distribution for ARPU
        'session_duration': session_duration_dist[n_control:],
        'device_type': device_type_dist[n_control:],
        'traffic_source': traffic_source_dist[n_control:],
        'region': region_dist[n_control:],
        'visitor_type': visitor_type_dist[n_control:]
    }

    # Convert to DataFrame
    df_control = pd.DataFrame(control_data)
    df_experiment = pd.DataFrame(experiment_data)
    
    # Combine control and experiment groups
    df = pd.concat([df_control, df_experiment], ignore_index=True)
    
    # Save the dataset
    df.to_csv('rimi_ab_test.csv', index=False)

    return df

# Setting parameters for control and experiment groups
n_control = 2790
n_experiment = 2790
start_date = '2024-08-05'
end_date = '2024-08-14'
control_conversion_rate = 0.04  # Conversion probability for the control group
experiment_conversion_rate = 0.046  # Conversion probability for the experiment group
control_arpu_mean = 50  # Average ARPU for the control group
experiment_arpu_mean = 55  # Average ARPU for the experiment group

# Generate data
df = generate_ab_test_data(n_control, n_experiment, start_date, end_date, control_conversion_rate, experiment_conversion_rate, control_arpu_mean, experiment_arpu_mean)

# Example of the first rows of the dataset
df.head()


,user_id,group,session_date,product_views,cart_adds,purchase_amount,session_duration,device_type,traffic_source,region,visitor_type
0,1,control,2024-08-09 06:00:00,6,0,0.0,26.378005,mobile,organic,Lithuania,old
1,2,control,2024-08-10 13:00:00,1,1,0.0,7.664598,mobile,paid_ad,Estonia,new
2,3,control,2024-08-10 15:00:00,6,2,0.0,6.676027,mobile,organic,Lithuania,new
3,4,control,2024-08-09 07:00:00,3,1,0.0,4.944476,desktop,paid_ad,Estonia,old
4,5,control,2024-08-05 23:00:00,6,2,0.0,4.328582,mobile,organic,Lithuania,old
